<a href="https://colab.research.google.com/github/yalopez84/Interpretable_method/blob/master/Interpretable_Triple_Prediction_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence_transformers
!pip install umap-learn 
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.2 MB/s 
     |████████████████████████████████| 4.4 MB 47.3 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 596 kB 68.2 MB/s 
     |████████████████████████████████| 6.6 MB 61.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=84d3e92ba78bb63f98744215ced31a5bb66944573da889af290f46ee8a9f0b98
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

In [3]:
from sentence_transformers import SentenceTransformer
import os
import csv
import sys
import random
from tqdm import tqdm, trange
import logging
import pdb
import umap
import hdbscan
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
from torch.utils.tensorboard import SummaryWriter
writter = SummaryWriter(log_dir='tensorboard_embedding_custom2')
logger = logging.getLogger(__name__)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pathFile="/content/drive/MyDrive/Almacen_phd/FB13/"
modelName="/content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2"
embeddings_large_file="/content/drive/MyDrive/Almacen_phd/FB13/embedding_large.pt"
umap_embeddings_file="/content/drive/MyDrive/Almacen_phd/FB13/umap_embeddings.pt"
asignacion_clusters="/content/drive/MyDrive/Almacen_phd/FB13/clusters_labels"
os.chdir(pathFile)


# Preprocessing-Start

In [6]:
class InputExample(object):

    def __init__(self, guid, text_a, text_b=None, text_c=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label

In [7]:
class Triple(object):
    def __init__(self, guid, subject , predicate , obj, label):
        self.guid=guid
        self.subject=subject
        self.predicate=predicate
        self.obj=obj
        self.label=label

In [8]:
class ExampleTruncado(object):

    def __init__(self, text, label=None):
        self.text = text
        self.label = label

In [9]:
class DataProcessor(object):
  
    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self, data_dir):
        raise NotImplementedError()

    def get_train_with_negatives(self,data_dir):
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []            
            for line in reader:
                lines.append(line)
            return lines
            
    @classmethod
    def _read_train_with_negatives(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter=" ", quotechar=quotechar)
            lines = []
            for line in reader:         
                lines.append(line)
            return lines

In [10]:
class KGProcessor(DataProcessor):
    def __init__(self):
        self.labels = set()
    
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train", data_dir)
        
    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "test_reduced.tsv")), "test", data_dir)
        
    def get_train_with_negatives(self,data_dir):
        return self._create_examples_with_negatives(
          self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt")), "embeddings", data_dir)

    def get_relations(self, data_dir):
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_labels(self, data_dir):
        return ["0", "1"]

    def get_entities(self, data_dir):
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    def get_prediction_test_triples(self, data_dir):
        with open(os.path.join(data_dir, "test_results.txt"), 'r') as f:
            lines = f.readlines()
            predictions = []
            for line in lines:               
                predictions.append(line.split()[2])
        return predictions

    def get_train_triples(self, data_dir):
        return self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt"))    

    def get_dev_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "test_reduced.tsv"))

    def _create_examples(self, lines, set_type, data_dir):
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        lines_str_set = set(['\t'.join(line) for line in lines])
        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if line[0] or line[2] not in auxiliar:
                head_ent_text = ent2text[line[0]]
                tail_ent_text = ent2text[line[2]]
                relation_text = rel2text[line[1]]
                flag_aux=True
            else:
                head_ent_text = line[0]
                tail_ent_text = line[2]
                relation_text = line[1]
                flag_aux=True

            if set_type == "dev" or set_type == "test" and flag_aux:
                triple_label = line[3]
                if triple_label == "1":
                    label = "1"
                else:
                    label = "0"

                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                self.labels.add(label)
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label=label))
                
            elif set_type == "train" and flag_aux:
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label="1"))

                rnd = random.random()
                guid = "%s-%s" % (set_type + "_corrupt", i)
                if rnd <= 0.5:
                    # corrupting head
                    tmp_head = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[0])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_head = random.choice(tmp_ent_list)
                        tmp_triple_str = tmp_head + '\t' + line[1] + '\t' + line[2]
                        if tmp_triple_str not in lines_str_set:
                            break                    
                    tmp_head_text = ent2text[tmp_head]
                    examples.append(
                        InputExample(guid=guid, text_a=tmp_head_text, text_b=text_b, text_c = text_c, label="0"))       
                else:
                    # corrupting tail
                    tmp_tail = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[2])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_tail = random.choice(tmp_ent_list)
                        tmp_triple_str = line[0] + '\t' + line[1] + '\t' + tmp_tail
                        if tmp_triple_str not in lines_str_set:
                            break
                    tmp_tail_text = ent2text[tmp_tail]
                    examples.append(
                        InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = tmp_tail_text, label="0"))      
                                                            
        return examples

    def _create_examples_with_negatives(self, lines, set_type, data_dir):
        
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if len(line)==5:
                if line[1] or line[3] not in auxiliar:               
                    guid=line[0]
                    head_ent_text = ent2text[line[1]]                
                    relation_text = rel2text[line[2]]
                    tail_ent_text = ent2text[line[3]]
                    triple_label = line[4]
                    examples.append(
                        InputExample(guid=guid, text_a=head_ent_text, text_b=relation_text, text_c = tail_ent_text, label=triple_label))     
                                                           
        return examples

In [11]:
def convert_examples_to_examplesTruncados(examples, max_seq_length, label_list, tokenizer=None, print_info = True):
    label_map = {label : i for i, label in enumerate(label_list)}
    examples_truncados = []
    logger.info("")
    logger.info("Writing examples truncados")
    logger.info("max_seq_length %d" % (max_seq_length))
    separador="////////////// "

    for (ex_index, example) in tqdm(enumerate(examples)):
        
        tokens_a = example.text_a.split()
        tokens_b = None
        tokens_c = None
       
        if example.text_b and example.text_c:
            tokens_b = example.text_b.split()
            tokens_c = example.text_c.split()
           
            _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_seq_length)
        else:
            if len(tokens_a) > max_seq_length:
                tokens_a = tokens_a[:(max_seq_length)]
        tokens =tokens_a.copy()

        if tokens_b:
            tokens.extend(tokens_b)           
        if tokens_c:
            tokens.extend(tokens_c)        

        padding = ["0"] * (max_seq_length - len(tokens))
        tokens.extend(padding)
        assert len(tokens) == max_seq_length

        examples_truncados.append(
                ExampleTruncado(text=tokens,label=example.label))
    return examples_truncados

In [12]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        sujeto=tokens_a.split()
        relation=tokens_b.split()
        total_length = len(sujeto) + len(relation)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [13]:
def _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b) + len(tokens_c)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b) and len(tokens_a) > len(tokens_c):
            tokens_a.pop()
        elif len(tokens_b) > len(tokens_a) and len(tokens_b) > len(tokens_c):
            tokens_b.pop()
        elif len(tokens_c) > len(tokens_a) and len(tokens_c) > len(tokens_b):
            tokens_c.pop()
        else:
            tokens_c.pop()

# Preprocessing-End

In [14]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt = '%m/%d/%Y %H:%M:%S',
    level = logging.INFO)

processor = KGProcessor()  
label_list = processor.get_labels(pathFile)

In [15]:
#Test triples
ids_list=[]
subjets_list=[]
predicates_list=[]
objects_list=[]
labels_list=[]
_examples_test = processor.get_test_examples(pathFile)
for sample in _examples_test:
    ids_list.append(sample.guid)
    subjets_list.append(sample.text_a)
    predicates_list.append(sample.text_b)
    objects_list.append(sample.text_c)
    labels_list.append(sample.label)

data_ids=pd.Series(ids_list)
data_subjects=pd.Series(subjets_list)
data_predicates=pd.Series(predicates_list)
data_objects=pd.Series(objects_list)
data_labels=pd.Series(labels_list)

frame_triples_test=pd.DataFrame({'Id_Triple_Test':data_ids,'Subjects':data_subjects,'Predicates':data_predicates, "Objects":data_objects, "Label":data_labels})
kg_Bert_predictions_list=processor.get_prediction_test_triples(os.path.join(pathFile,"output_FB13" ))
frame_triples_test["KG BERT Prediction"]=kg_Bert_predictions_list
frame_triples_test

100%|██████████| 75042/75042 [00:00<00:00, 192203.60it/s]


,Id_Triple_Test,Subjects,Predicates,Objects,Label,KG BERT Prediction
0,test-0,Umberto I (Italian: Umberto Rainerio Carlo Ema...,cause of death,Tyrannicide is the killing or assassination of...,1,1
1,test-1,"John Glenn Beall Jr. (June 19, 1927 – March 24...",nationality,"The United States of America (USA), commonly k...",1,1
2,test-2,John Atkinson Grimshaw (6 September 1836 – 13 ...,gender,A male (♂) organism is the physiological sex t...,1,1
3,test-3,"Henry John Heinz (October 11, 1844 – May 14, 1...",nationality,"Germany (German: Deutschland, German pronuncia...",1,1
4,test-4,"Ernest Lundeen (August 4, 1878 – August 31, 19...",profession,A politician is a person active in party polit...,1,1
...,...,...,...,...,...,...
2248,test-2248,"Lew Cody (born Louis Joseph Côté; February 22,...",gender,A male (♂) organism is the physiological sex t...,0,1
2249,test-2249,Leo Szilard (; Hungarian: Szilárd Leó [ˈsilaːr...,profession,A laborer or labourer is a person who works in...,0,0
2250,test-2250,Quentin Crisp (born Denis Charles Pratt; (1908...,profession,"A disc jockey, more commonly abbreviated as DJ...",0,0
2251,test-2251,Tobias Mayer (17 February 1723 – 20 February 1...,nationality,"East Germany, officially the German Democratic...",0,1


In [16]:
#Train triples
ids_list2=[]
subjets_list2=[]
predicates_list2=[]
objects_list2=[]
labels_list2=[]
_examples_train=processor.get_train_with_negatives(pathFile) 
for sample2 in _examples_train:
    ids_list2.append(sample2.guid)
    subjets_list2.append(sample2.text_a)
    predicates_list2.append(sample2.text_b)
    objects_list2.append(sample2.text_c)
    labels_list2.append(sample2.label)

data2_ids=pd.Series(ids_list2)
data2_subjects=pd.Series(subjets_list2)
data2_predicates=pd.Series(predicates_list2)
data2_objects=pd.Series(objects_list2)
data2_labels=pd.Series(labels_list2)

frame_triples_train=pd.DataFrame({'Id_Triple_Training':data2_ids, 'Subjects':data2_subjects,'Predicates':data2_predicates, "Objects":data2_objects, "Label":data2_labels})
frame_triples_train

100%|██████████| 75042/75042 [00:00<00:00, 202952.68it/s]


,Id_Triple_Training,Subjects,Predicates,Objects,Label
0,train-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Cancer is a group of diseases involving abnorm...,1
1,train_corrupt-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Thomas Langlois Lefroy (8 January 1776 – 4 May...,0
2,train-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"In aviation, an accident is defined by the Con...",1
3,train_corrupt-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"John Manners, 2nd Duke of Rutland KG (18 Septe...",0
4,train-2,Rajiv Ratna Gandhi ( (listen); 20 August 1944 ...,cause of death,Assassination is the act of killing a prominen...,1
...,...,...,...,...,...
22159,train_corrupt-11079,"Irkutsk (Russian: Иркутск, IPA: [ɪrˈkutsk]; Bu...",profession,"Engineers, as practitioners of engineering, ar...",0
22160,train-11080,Radhanath Sikdar (Bengali: রাধানাথ শিকদার; Oct...,profession,A mathematician is someone who uses an extensi...,1
22161,train_corrupt-11080,"Archduke Karl Salvator of Austria, in Italian ...",profession,A mathematician is someone who uses an extensi...,0
22162,train-11081,Peter Joseph William Debye (; Dutch: [dəˈbɛiə...,profession,A scientist is someone who conducts scientific...,1


In [17]:
#Select the test triple to be explained
id_input_triple=input("Enter the test triple id to be explained:")
frame_triples_test[frame_triples_test["Id_Triple_Test"]==id_input_triple]

Enter the test triple id to be explained:test-1


,Id_Triple_Test,Subjects,Predicates,Objects,Label,KG BERT Prediction
1,test-1,"John Glenn Beall Jr. (June 19, 1927 – March 24...",nationality,"The United States of America (USA), commonly k...",1,1


In [18]:
#Turn training triples + target test triple into sequences 
_examples_train_truncates= convert_examples_to_examplesTruncados(_examples_train, 512,label_list)
X_train=[]
X_train2=[]
for sample in _examples_train_truncates:
    X_train.append(" ".join(sample.text))

target_triple_test=list(e for e in _examples_test if e.guid  == id_input_triple)
target_triple_sequence_truncate=convert_examples_to_examplesTruncados(target_triple_test, 512,label_list)
X_train2.append(" ".join(target_triple_sequence_truncate[0].text))

07/18/2022 19:25:22 - INFO - __main__ -   
07/18/2022 19:25:22 - INFO - __main__ -   Writing examples truncados
07/18/2022 19:25:22 - INFO - __main__ -   max_seq_length 512
22164it [00:02, 7819.67it/s]
07/18/2022 19:25:26 - INFO - __main__ -   
07/18/2022 19:25:26 - INFO - __main__ -   Writing examples truncados
07/18/2022 19:25:26 - INFO - __main__ -   max_seq_length 512
1it [00:00, 1225.33it/s]


In [19]:
#Get triple embeddings by a Sentence Transformer model
model = SentenceTransformer(modelName)
if os.path.exists(embeddings_large_file):
    embeddings_large=torch.load(embeddings_large_file)
else:
    embeddings_large = model.encode(X_train, show_progress_bar=True) 
    torch.save(embeddings_large,embeddings_large_file) 

embeddings_large_target_triple=model.encode(X_train2, show_progress_bar=True) 
embeddings_large=np.append(embeddings_large,embeddings_large_target_triple, axis=0)
pd.DataFrame(embeddings_large).head()

07/18/2022 19:25:26 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: /content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2
07/18/2022 19:25:35 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.124780,0.384982,-0.003927,-0.395087,0.287894,0.457958,-0.039695,0.219512,0.018069,-0.021312,...,-0.007924,-0.008788,-0.107553,0.294230,0.432136,-0.274154,-0.148381,-0.362436,-0.201350,-0.197279
1,-0.782613,-0.228494,0.476549,-0.103342,-0.334109,0.372395,-0.154412,-0.503228,-0.135285,-0.002706,...,0.519501,-0.198084,-0.054243,-0.149871,0.623927,-1.093331,-0.501036,-0.394166,0.135077,0.490513
2,-0.489224,0.157877,0.181251,-0.003547,-0.183236,-0.058668,0.112045,-0.526372,0.108750,-0.069467,...,0.306203,0.448789,-0.029652,-0.743275,0.008782,-0.453773,-0.342212,-0.207790,0.144582,0.136782
3,-0.746945,0.051390,0.535060,0.104407,-0.528381,0.009819,-0.156311,-0.688953,0.078460,-0.121379,...,0.174695,0.100970,-0.045806,-0.189165,0.479870,-0.850080,-0.778050,-0.448173,0.422841,0.801237
4,-0.400677,-0.634836,-0.385898,-0.033849,-0.595452,0.537820,0.008352,-0.127600,-0.200405,0.363751,...,-0.059161,0.486053,0.067214,-0.040830,0.339542,-0.403921,-0.199044,-0.209626,-0.000264,-0.127599


In [20]:
#Dimension reduction with UMAP
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5,
                            min_dist=0.0,
                            metric='cosine').fit_transform(embeddings_large)
torch.save(umap_embeddings, umap_embeddings_file)
pd.DataFrame(umap_embeddings)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


,0,1,2,3,4
0,6.417902,8.065788,5.377629,-2.311454,-2.333835
1,10.415637,6.418827,8.131248,4.929911,0.418924
2,8.045989,4.510033,9.121522,4.322084,1.901458
3,10.104101,5.630599,8.023499,5.206674,0.369550
4,9.597733,11.427177,9.920119,2.458481,0.313887
...,...,...,...,...,...
22160,11.065282,5.552539,9.453191,5.093204,0.479086
22161,11.033111,5.558297,9.381298,5.111158,0.451011
22162,10.935255,6.592891,9.545259,4.610346,0.712565
22163,10.942450,6.591733,9.560614,4.613143,0.720847


In [21]:
#To cluster embeddings with HDBSCAN
from sklearn.preprocessing import normalize
umap_embeddings_normalized=normalize(umap_embeddings, norm='l2')
cluster = hdbscan.HDBSCAN(min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='eom'
                        ).fit(umap_embeddings_normalized)

#Saving triples with their cluster labels
with open(asignacion_clusters, "w") as writer:
    writer.write("Clusters asignados\n")
    writer.write("id de tripleta -------cluster\n")
    for id, idcluster in enumerate(cluster.labels_):
        writer.write("%d ---------- %d\n" % (id, idcluster))    

print(f"Number of clusters = {np.unique(cluster.labels_).shape[0] - 1}") 

Number of clusters = 610


In [22]:
#Getting the positions of triples in the same cluster of the target triple
cluster_target=cluster.labels_[len(cluster.labels_)-1]
position_triples_targets=[pos for pos,value in enumerate(cluster.labels_) if value==cluster_target]

In [23]:
#Target triple set
target_triples_set = [example for pos, example in enumerate(_examples_train) if pos in position_triples_targets]
ids_list3=[]
subjets_list3=[]
predicates_list3=[]
objects_list3=[]
labels_list3=[]
for sample3 in target_triples_set:
    ids_list3.append(sample3.guid)
    subjets_list3.append(sample3.text_a)
    predicates_list3.append(sample3.text_b)
    objects_list3.append(sample3.text_c)
    labels_list3.append(sample3.label)

data3_ids=pd.Series(ids_list3)
data3_subjects=pd.Series(subjets_list3)
data3_predicates=pd.Series(predicates_list3)
data3_objects=pd.Series(objects_list3)
data3_labels=pd.Series(labels_list3)
frame3_target_triple_train=pd.DataFrame({'Id_triple_Training':data3_ids, 'Subjects':data3_subjects,'Predicates':data3_predicates, "Objects":data3_objects, "Label":data3_labels})
frame3_target_triple_train

,Id_triple_Training,Subjects,Predicates,Objects,Label
0,train_corrupt-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Thomas Langlois Lefroy (8 January 1776 – 4 May...,0
1,train_corrupt-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"John Manners, 2nd Duke of Rutland KG (18 Septe...",0
2,train_corrupt-5,Murder in Small Town X (sometimes abbreviated ...,cause of death,Sanjay Gandhi (14 December 1946 – 23 June 1980...,0
3,train_corrupt-13,"Andrés Bonifacio y de Castro (November 30, 186...",cause of death,Karl Ferdinand Gutzkow ((1811-03-17)17 March 1...,0
4,train_corrupt-15,"Polly Ann Young (October 25, 1908 – January 21...",cause of death,"Richard of Chichester (1197 – 3 April 1253), a...",0
...,...,...,...,...,...
6602,train-11076,"László Tisza (July 7, 1907 – April 15, 2009) w...",nationality,"The United States of America (USA), commonly k...",1
6603,train-11077,"William Sherman Jennings (March 24, 1863 – Feb...",nationality,"The United States of America (USA), commonly k...",1
6604,train-11078,"Joseph Edward Levine (September 9, 1905 – July...",nationality,"The United States of America (USA), commonly k...",1
6605,train_corrupt-11078,"Tiger ""Tige"" Andrews (March 19, 1920 – January...",nationality,"The United States of America (USA), commonly k...",0


In [24]:
#Target triple set embeddings
umap_embeddings_target_triple_set= [vector for pos, vector in enumerate(umap_embeddings[:-1]) if pos in position_triples_targets]
umap_embeddings_target_triple_set_df=pd.DataFrame(umap_embeddings_target_triple_set)

ids_triple_target_set= [sample.guid for pos, sample in enumerate(_examples_train) if pos in position_triples_targets]
labels_triple_target_set= [sample.label for pos, sample in enumerate(_examples_train) if pos in position_triples_targets]

umap_embeddings_target_triple_set_df["Id_triple_Training"]=ids_triple_target_set
umap_embeddings_target_triple_set_df["Label"]=labels_triple_target_set

umap_embeddings_target_triple_set_df.to_excel("Embeddings_Regresion.xlsx")
umap_embeddings_target_triple_set_df

,0,1,2,3,4,Id_triple_Training,Label
0,10.415637,6.418827,8.131248,4.929911,0.418924,train_corrupt-0,0
1,10.104101,5.630599,8.023499,5.206674,0.369550,train_corrupt-1,0
2,10.140753,5.924430,7.856790,5.445376,0.135439,train_corrupt-5,0
3,10.335453,6.139719,8.159469,5.273522,0.262583,train_corrupt-13,0
4,10.571750,5.713284,8.113235,5.439028,0.263786,train_corrupt-15,0
...,...,...,...,...,...,...,...
6602,9.503907,11.435890,10.419710,-3.095346,5.394457,train-11076,1
6603,9.562716,11.551041,10.265273,-3.185982,5.535586,train-11077,1
6604,9.638798,11.650510,10.430031,-3.294233,4.698323,train-11078,1
6605,9.666303,11.695898,10.361270,-3.365531,4.870100,train_corrupt-11078,0


In [25]:
#Triple target prediction by Logistic Regression
import math
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6]) 

features_list=["Feature_0", "Feature_1", "Feature_2", "Feature_3", "Feature_4"]
feature_name=pd.Series(features_list)
feature_values=lr.coef_[0]
odds_ratio=np.exp(feature_values)
frame_logistic_regression_interpretability=pd.DataFrame({'Features':feature_name, 'Weight':feature_values, 'Odds Ratio':odds_ratio })
frame_logistic_regression_interpretability
pred = lr.predict(umap_embeddings[-1,:].reshape(1,5))
pred
log_regresion=pred[0]
print("Logistic Regression prediction:", str(log_regresion))
frame_logistic_regression_interpretability


Logistic Regression prediction: 1


,Features,Weight,Odds Ratio
0,Feature_0,0.089160,1.093255
1,Feature_1,0.386133,1.471280
2,Feature_2,0.002732,1.002736
3,Feature_3,-0.067789,0.934457
4,Feature_4,0.027399,1.027778


In [26]:
#Triple target prediction by Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

dt = DecisionTreeClassifier(max_depth=5)
dt.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
pred = dt.predict(umap_embeddings[-1,:].reshape(1,5))
dec_tree_prediction=pred[0]
dec_tree_prediction

'1'

In [27]:
#Decision tree
from sklearn.tree import export_text
r = export_text(dt)
print(r)

|--- feature_1 <= 7.04
|   |--- feature_0 <= 10.89
|   |   |--- feature_0 <= 9.82
|   |   |   |--- feature_1 <= 6.51
|   |   |   |   |--- class: 0
|   |   |   |--- feature_1 >  6.51
|   |   |   |   |--- feature_1 <= 6.99
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_1 >  6.99
|   |   |   |   |   |--- class: 0
|   |   |--- feature_0 >  9.82
|   |   |   |--- feature_4 <= -1.48
|   |   |   |   |--- feature_4 <= -1.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_4 >  -1.50
|   |   |   |   |   |--- class: 1
|   |   |   |--- feature_4 >  -1.48
|   |   |   |   |--- feature_0 <= 10.66
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_0 >  10.66
|   |   |   |   |   |--- class: 0
|   |--- feature_0 >  10.89
|   |   |--- feature_3 <= 4.12
|   |   |   |--- feature_2 <= 8.69
|   |   |   |   |--- feature_2 <= 8.57
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_2 >  8.57
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_2 >  8.6

In [28]:
#Triple target prediction by KNN and influential triples embeddings (nearest neighbors)
from numpy.core.fromnumeric import squeeze
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=5) 
rnc = RadiusNeighborsClassifier(radius=5) 
knc.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
rnc.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
knc_pred = knc.predict(umap_embeddings[-1,:].reshape(1,5))
knn_predicion=knc_pred[0]
print("knn prediction:", knn_predicion)
_, knc_neighbors = knc.kneighbors(umap_embeddings[-1,:].reshape(1,5))
nearest_neighbors=knc_neighbors.squeeze().tolist()
nearest_neighbors_df=umap_embeddings_target_triple_set_df.iloc[nearest_neighbors]
print("Influential triples embeddings:")
nearest_neighbors_df

knn prediction: 1
Influential triples embeddings:


,0,1,2,3,4,Id_triple_Training,Label
1114,9.483561,11.548245,10.191760,-2.874357,5.418274,train-1722,1
6323,9.473804,11.567585,10.206584,-2.940132,5.442009,train-10492,1
6196,9.485258,11.569527,10.243593,-2.916224,5.370186,train-10240,1
997,9.470648,11.605691,10.192276,-2.933139,5.452052,train-1531,1
85,9.461874,11.545371,10.228652,-2.942273,5.414216,train-147,1


In [29]:
#Influential triples lines by KNN (nearest neighbors)
from numpy.ma.core import append
id_triples=nearest_neighbors_df.iloc[:,5].unique()
train_triples_lines=processor.get_train_triples(pathFile)
influential_triples=[]
train_triples_lines_neighbors= [triple for triple in train_triples_lines if triple[0] in id_triples]
for triple in train_triples_lines_neighbors:
    influential_triples.append(Triple(guid=triple[0],subject=triple[1],predicate=triple[2],obj=triple[3], label=triple[4]))

id_triple_target=int(id_input_triple.split("-")[1])
triple_target_line=processor.get_test_triples(pathFile)[id_triple_target]

kg_Bert_Precition_value=kg_Bert_predictions_list[id_triple_target]
log_reg_prediction_value=log_regresion
dec_tree_prediction_value=dec_tree_prediction
knn_predicion_value=knn_predicion
label=1 if triple_target_line[3]=="1" else 0
print('\033[1m'+"Triple target \033[0m:  {}   {}    {}   label:{}".format(triple_target_line[0], triple_target_line[1],triple_target_line[2], label))

print('\033[1m'+"Predictions" +'\033[0m'": KG BERT:{}    Log Regression:{}    Decision Tree:{}    knn:{}".format(kg_Bert_Precition_value, log_reg_prediction_value, dec_tree_prediction_value, knn_predicion_value))
print("\n")
print('\033[1m'+" Influential triples:"+'\033[0m')

#Influential triples for the target triple prediction
ids_list4=[]
subjets_list4=[]
predicates_list4=[]
objects_list4=[]
labels_list4=[]

for sample4 in influential_triples:
    ids_list4.append(sample4.guid)
    subjets_list4.append(sample4.subject)
    predicates_list4.append(sample4.predicate)
    objects_list4.append(sample4.obj)
    labels_list4.append(sample4.label)

data4_ids=pd.Series(ids_list4)
data4_subjects=pd.Series(subjets_list4)
data4_predicates=pd.Series(predicates_list4)
data4_objects=pd.Series(objects_list4)
data4_labels=pd.Series(labels_list4)

frame4_influential_triple_train=pd.DataFrame({'Id_triple_Training':data4_ids, 'Subjects':data4_subjects,'Predicates':data4_predicates, "Objects":data4_objects, "Label":data4_labels})
frame4_influential_triple_train


Triple target :  john_glenn_beall_jr   nationality    united_states   label:1
Predictions: KG BERT:1    Log Regression:1    Decision Tree:1    knn:1


 Influential triples:


,Id_triple_Training,Subjects,Predicates,Objects,Label
0,train-147,ed_pulaski,nationality,united_states,1
1,train-1531,james_rood_doolittle,nationality,united_states,1
2,train-1722,james_moore_wayne,nationality,united_states,1
3,train-10240,green_b_raum,nationality,united_states,1
4,train-10492,lynn_frazier,nationality,united_states,1
